Merge data from the geoJSON of the world map and data from the co2 emissions per capita 

In [2]:
import pandas as pd
import numpy as np
from pprint import pprint
import geojson
import json

In [3]:
# load world map data
with open('custom.geo.json') as geo:
    gj = geojson.load(geo)
features = gj['features']

# keep only useful properties
keep_properties = ['name','name_long','iso_a3','continent','label_x','label_y']

for k in reversed(features):
    properties = {}
    for label in keep_properties:
        properties[label] = k["properties"][label]
    k["properties"] = properties

In [4]:
# load dataframes
df = pd.read_csv('../../data/co-emissions-per-capita.csv')
df = df.replace(0.0, '')
df2 = pd.read_csv('../../data/share-above-who-pollution-guidelines.csv')
df2 = df2.replace(0.0, '')

In [5]:
# rename countries to match geoJSON
oldNames = [
    'Czechia',
    'Russia',
    'Gambia',
    'Eswatini',
    'Cote d\'Ivoire',
    'Congo',
    'Democratic Republic of Congo',
    'Brunei',
    'Timor',
    'Laos',
    'North Korea',
    'South Korea',
    'Falkland Islands',
]
newNames = [
    'Czech Republic',
    'Russian Federation',
    'The Gambia',
    'Kingdom of eSwatini',
    'Côte d\'Ivoire',
    'Republic of the Congo',
    'Democratic Republic of the Congo',
    'Brunei Darussalam',
    'Timor-Leste',
    'Lao PDR',
    'Dem. Rep. Korea',
    'Republic of Korea',
    'Falkland Islands / Malvinas',
]

df = df.replace(oldNames,newNames)
df2 = df2.replace(oldNames,newNames)

In [6]:
# fill geomap data with emissions

print('## co2 unmatched countries:')

for k in reversed(features):
    country_name = k["properties"]["name_long"]
    country_data = df.loc[df['Entity'] == country_name]
    if country_data.empty:
        print(country_name)
    else:
        # create dict {year: emission}
        emissions = dict(zip(country_data["Year"], country_data["Annual CO₂ emissions (per capita)"]))
        k["properties"]["emitted_co2"] = emissions

print('## air unmatched countries:')

for k in reversed(features):
    country_name = k["properties"]["name_long"]
    country_data = df2.loc[df2['Entity'] == country_name]
    if country_data.empty:
        print(country_name)
    else:
        # create dict {year: emission}
        emissions = dict(zip(country_data["Year"], country_data["PM2.5 air pollution, population exposed to levels exceeding WHO guideline value (% of total)"]))
        k["properties"]["air_pollution"] = emissions


## co2 unmatched countries:
Western Sahara
Somaliland
Northern Cyprus
Falkland Islands / Malvinas
## air unmatched countries:
New Caledonia
Kosovo
Western Sahara
Somaliland
Taiwan
Northern Cyprus
Falkland Islands / Malvinas


In [7]:
with open('co2_air.json', 'w') as outfile:
    json.dump(gj, outfile)